In [1]:
#%load_ext sql
%reload_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql SELECT * FROM employees

 * sqlite:///chinook.db
Done.


EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


## Задание 1
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Rock, приобретенные сотрудниками Microsoft.


In [27]:
%%sql
SELECT ar.Name AS ArtistName, al.Title AS AlbumTitle, t.Name AS TrackName
  FROM invoice_items ii
LEFT JOIN tracks t
  ON ii.TrackId = t.TrackId
LEFT JOIN genres g
  ON t.GenreId = g.GenreId
LEFT JOIN albums al
  ON al.AlbumId = t.AlbumId
LEFT JOIN artists ar
  ON al.ArtistId = ar.ArtistId
LEFT JOIN invoices i
  ON ii.InvoiceId = i.InvoiceId
LEFT JOIN Customers c
  ON i.CustomerId = c.CustomerId
  WHERE c.Company LIKE "Microsoft%" AND g.Name LIKE "Rock";

 * sqlite:///chinook.db
Done.


ArtistName,AlbumTitle,TrackName
Guns N' Roses,Use Your Illusion I,Double Talkin' Jive
Guns N' Roses,Use Your Illusion I,The Garden
Guns N' Roses,Use Your Illusion I,Don't Damn Me
Guns N' Roses,Use Your Illusion I,Dead Horse
Foo Fighters,In Your Honor [Disc 1],No Way Back
Foo Fighters,In Your Honor [Disc 2],Still
Foo Fighters,In Your Honor [Disc 2],Razor
Foo Fighters,The Colour And The Shape,My Hero


## Задание 2
Вывести 10 самых больших по размеру треков жанра Blues и формата MPEG (используйте команду LIMIT 10). Только название и автор.


In [36]:
%%sql
SELECT ar.Name AS ArtistName, t.Name AS TrackName
  FROM tracks t
LEFT JOIN genres g
  ON t.GenreId = g.GenreId
LEFT JOIN media_types mt
  ON t.MediaTypeId = mt.MediaTypeId
LEFT JOIN albums al
  ON al.AlbumId = t.AlbumId
LEFT JOIN artists ar
  ON al.ArtistId = ar.ArtistId
  WHERE g.Name LIKE "Blues" AND mt.Name LIKE "MPEG%"
  ORDER BY t.Bytes DESC
  LIMIT 10;

 * sqlite:///chinook.db
Done.


ArtistName,TrackName
Buddy Guy,Talkin' 'Bout Women Obviously
Stevie Ray Vaughan & Double Trouble,Riviera Paradise
The Black Crowes,Title Song
Eric Clapton,Old Love
The Black Crowes,Wiser Time
Buddy Guy,Stone Crazy
Eric Clapton,Layla
The Black Crowes,Girl From A Pawnshop
The Black Crowes,Sometimes Salvation
Eric Clapton,Running On Faith


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций.


In [74]:
%%sql

WITH companies AS ( SELECT c.Company as Name
    FROM customers c
    WHERE c.Company IS NOT NULL
    GROUP BY c.Company
  )

SELECT co.Name as CompanyName,
  (
    SELECT COALESCE(SUM(ii.Quantity),0)
      FROM invoices i
    LEFT JOIN customers cu
      ON i.CustomerId = cu.CustomerId
    LEFT JOIN invoice_items ii
      ON ii.InvoiceId = i.InvoiceId
    LEFT JOIN tracks t
      ON ii.TrackId = t.TrackId
    LEFT JOIN genres g
      ON t.GenreId = g.GenreId
      WHERE co.Name = cu.Company AND g.Name LIKE "Jazz"
  ) as BoughtTracks
  FROM
    companies co;

 * sqlite:///chinook.db
Done.


CompanyName,BoughtTracks
Apple Inc.,2
Banco do Brasil S.A.,0
Embraer - Empresa Brasileira de Aeronáutica S.A.,0
Google Inc.,2
JetBrains s.r.o.,3
Microsoft Corporation,2
Riotur,0
Rogers Canada,0
Telus,2
Woodstock Discos,0


## Задание 4
Вывести топ 3 альбома по продолжительности (продолжительность альбома - сумма длин треков в нём). Только название и продолжительность.


In [86]:
%%sql
SELECT a1.Title AS AlbumTitle, (
    SELECT COALESCE(SUM(t.Milliseconds),0)
      FROM tracks t
    LEFT JOIN albums a2
      ON t.AlbumId = a2.AlbumId
      WHERE a1.AlbumId = a2.AlbumId
  ) as AlbumLength
  FROM albums a1
  ORDER BY AlbumLength DESC
  LIMIT 3;

 * sqlite:///chinook.db
Done.


AlbumTitle,AlbumLength
"Lost, Season 3",70665582
"Battlestar Galactica (Classic), Season 1",70213784
"Lost, Season 1",64854936


## Задание 5
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и их общее количество, причем вывести только те наборы, для которых **все** треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек.

In [100]:
%%sql

SELECT DISTINCT g.Name AS GenreName, mt.Name AS MediaTypeName,
  (
    SELECT ROUND(AVG(ii.UnitPrice),2)
      FROM invoice_items ii
    LEFT JOIN tracks t2
      ON ii.TrackId = t2.TrackId
      WHERE t2.GenreId = g.GenreId AND t2.MediaTypeId = mt.MediaTypeId AND ii.UnitPrice > 1.5
      GROUP BY t2.TrackId
  ) AS AveragePrice,
  (
    SELECT COUNT(*)
      FROM tracks t2
      WHERE t2.GenreId = g.GenreId AND t2.MediaTypeId = mt.MediaTypeId AND ii.UnitPrice > 1.5
  ) as TracksCount
  FROM (genres g, media_types mt)
LEFT JOIN tracks t1
  ON t1.GenreId = g.GenreId AND t1.MediaTypeId = mt.MediaTypeId
  WHERE AveragePrice IS NOT NULL

 * sqlite:///chinook.db
(sqlite3.OperationalError) no such column: ii.UnitPrice
[SQL: SELECT DISTINCT g.Name AS GenreName, mt.Name AS MediaTypeName,
  (
    SELECT ROUND(AVG(ii.UnitPrice),2)
      FROM invoice_items ii
    LEFT JOIN tracks t2
      ON ii.TrackId = t2.TrackId
      WHERE t2.GenreId = g.GenreId AND t2.MediaTypeId = mt.MediaTypeId AND ii.UnitPrice > 1.5
      GROUP BY t2.TrackId
  ) AS AveragePrice,
  (
    SELECT COUNT(*)
      FROM tracks t2
      WHERE t2.GenreId = g.GenreId AND t2.MediaTypeId = mt.MediaTypeId AND ii.UnitPrice > 1.5
  ) as TracksCount
  FROM (genres g, media_types mt)
LEFT JOIN tracks t1
  ON t1.GenreId = g.GenreId AND t1.MediaTypeId = mt.MediaTypeId
  WHERE AveragePrice IS NOT NULL]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
